# Utiliser l'API Google Books pour récupérer des données de livres

## Partie 1 : Comprendre les API

### a. Qu’est-ce qu’une API ?
- API = Application Programming Interface
- Une API est une interface qui permet à un programme d’accéder aux fonctionnalités ou aux données d’un autre service, sans avoir besoin de connaître son fonctionnement interne.
- Exemple : l’API Google Books met à disposition des données sur des millions de livres, que l’on peut interroger depuis un programme Python.
- On envoie une **requête** (question) à l’API, elle nous répond avec des **données** (souvent en JSON)

### b. Qu’est-ce qu’une requête HTTP ?
- HTTP = HyperText Transfer Protocol – c’est le protocole qui permet à des machines (navigateur, script Python, application mobile...) de communiquer avec des serveurs sur Internet.
- Une requête HTTP est un message envoyé depuis un client (comme un navigateur ou un programme Python) vers un serveur web pour lui demander quelque chose.
- Quand on ouvre un site web ou que l'on demande des données à une API, on envoie une **requête HTTP**, généralement de type :

    - **GET** → pour demander des données
    - **POST** → pour envoyer des données
    - **PUT**, **DELETE**, etc.

- L’API Google Books utilise principalement des requêtes de type GET, comme si on entrait une URL dans un navigateur pour demander des infos.
- Une URL de requête API suit une structure bien définie :

`<URL de base> ? <paramètre1>=<valeur1> & <paramètre2>=<valeur2> & ...`

- URL de base : adresse du point d’entrée de l’API, aussi appelée **endpoint**.
- ? : sépare l’URL de base des paramètres de requête.
- Paramètres de requête (query parameters) : ajoutés sous forme de paires clé=valeur. Chaque paramètre est séparé par &.  Les paramètres possibles sont disponibles dans la documentation de chaque API.

### c. Réponse

Quand on envoie une requête GET à une API, on demande au serveur de renvoyer une ressource.

La réponse de ce serveur comporte deux parties essentielles :


1. **Le code de status de réponse (HTTP status code)**

C’est un nombre à 3 chiffres qui indique si la requête a réussi ou non.

--> [Liste des codes possibles](https://developer.mozilla.org/fr/docs/Web/HTTP/Reference/Status) et leur signification


2. **Le contenu de la réponse (le "corps")**

C’est ici que se trouvent les données demandées.

Ce contenu est souvent au format JSON (JavaScript Object Notation), c’est-à-dire une structure clé-valeur que Python peut facilement lire.

Exemple d'un format JSON :
```json
{
  "items": [
    {
      "tripInfo": {
        "country": "France",
        "cities": ["Montpellier", "Marseille"],
        "nb_photo_taken": 200,
        "language": "fr"
      }
    }
  ]
}
```

---
## Partie 2 : Requêter Google Books API

Dans cette partie vous allez : 
- Faire une requête HTTP pour interroger l'API Google Books à l'aide de la bibliothèque python _requests_,
- Vérifier le code de status de la réponse,
- Traiter le corps de la réponse à l'aide de la bibliothèque python _json_,
- Utiliser _pandas_ pour convertir les données reçues en dataframe.


In [2]:
# Importer les bibliothèques nécessaires
import requests
import json
import pandas as pd

Pour interroger une API, un certain nombre de paramètres et filtres définis dans la documentation peuvent être utilisés. 

En utilisant la [documentation](https://developers.google.com/books/docs/v1/using?hl=fr#WorkingVolumes) de l'API Google Books, définir le dictionnaire de paramètres permettant de faire une recherche sur :
- la chaîne de texte : "food",
- il doit s'agir d'un e-book avec un prix d'achat,
- le nombre maximal de résultats à renvoyer est de 40,
- les résultats doivent être triés par pertinence.

In [28]:
# URL de l'API Google Books
url = "https://www.googleapis.com/books/v1/volumes"

# Dictionnaire de paramètres pour la requête

payload = {"q":"food","filter":"paid-ebooks","maxResults":40,"orderBy":"relevance"}

En utilisant la documentation de la bibliothèque [_requests_](https://requests.readthedocs.io/en/latest/user/quickstart/):
- implémenter la requête HTTP _get_  avec les paramètres sur l'URL définie ci-dessus,
- vérifier le code de réponse HTTP.

In [29]:
# Requêter l'API Google Books
response = requests.get(url, params=payload)
print(response.url)

# Vérifier le code de statut de la réponse
response.status_code

https://www.googleapis.com/books/v1/volumes?q=food&filter=paid-ebooks&maxResults=40&orderBy=relevance


200

Récupérer et observer le coeur de la réponse via l'objet _response_ créé précédemment.

In [134]:
# Récupérer le coeur de la réponse
data_books_raw = response.json()

# Afficher les données récupérées

print(json.dumps(data_books_raw, indent=2))

# Afficher le type de la variable data_books_raw
print(type(data_books_raw))


{
  "kind": "books#volumes",
  "totalItems": 1000000,
  "items": [
    {
      "kind": "books#volume",
      "id": "sa6pDAAAQBAJ",
      "etag": "d5b6fE3+Iw8",
      "selfLink": "https://www.googleapis.com/books/v1/volumes/sa6pDAAAQBAJ",
      "volumeInfo": {
        "title": "Food",
        "subtitle": "A Culinary History",
        "authors": [
          "Jean-Louis Flandrin",
          "Massimo Montanari"
        ],
        "publisher": "Columbia University Press",
        "publishedDate": "1999-11-23",
        "description": "When did we first serve meals at regular hours? Why did we begin using individual plates and utensils to eat? When did \"cuisine\" become a concept and how did we come to judge food by its method of preparation, manner of consumption, and gastronomic merit? Food: A Culinary History explores culinary evolution and eating habits from prehistoric times to the present, offering surprising insights into our social and agricultural practices, religious beliefs, and m

- Les données relatives aux livres récupérées sont accessibles dans la clé 'items',
- Récupérer les données relatives aux livres dans la variable data_books.

**NOTE**

Il existe 2 méthodes pour accéder aux valeurs d'une clé ("nom_de_la_clé") d'un dictionnaire python : 
- `mon_dict["nom_de_la_clé"]`: si "nom_de_la_clé" n'est pas présent dans le dictionnaire, cela retourne une erreur.
- `mon_dict.get("nom_de_la_clé")` : si "nom_de_la_clé" n'est pas présent dans le dictionnaire, cela renvoie None par défault. 
Il est aussi possible d'indiquer comme second argument la variable à renvoyer si la clé n'est pas présente : 

`mon_dict.get("nom_de_la_clé", "clé inexistante")`

Privilégier la seconde méthode car certaines clés peuvent ne pas exister.

--> [Lien ressource](https://w3schools.tech/fr/tutorial/python/python_access_dictionary_items) : accès au clé/valeurs d'un dictionnaire python

In [42]:
# Récupérer les données relatives aux livres via la clé 'items'
data_books = data_books_raw.get("items", "clé inexistante")

# Afficher le type de la variable data_books
print(type(data_books))


<class 'list'>


Accéder au premier élément de _data_books_ pour afficher les données du premier livre.

--> [Lien ressource](https://w3schools.tech/fr/tutorial/python/python_access_list_items) : accéder aux éléments d'une liste python

In [ ]:
# Afficher le premier livre
print(data_books[0])

{'title': 'Food', 'subtitle': 'A Culinary History', 'authors': ['Jean-Louis Flandrin', 'Massimo Montanari'], 'publisher': 'Columbia University Press', 'publishedDate': '1999-11-23', 'industryIdentifiers': [{'type': 'ISBN_13', 'identifier': '9780231544092'}, {'type': 'ISBN_10', 'identifier': '023154409X'}], 'readingModes': {'text': True, 'image': False}, 'pageCount': 644, 'printType': 'BOOK', 'categories': ['Cooking'], 'maturityRating': 'NOT_MATURE', 'allowAnonLogging': True, 'contentVersion': '1.11.9.0.preview.2', 'panelizationSummary': {'containsEpubBubbles': False, 'containsImageBubbles': False}, 'imageLinks': {'smallThumbnail': 'http://books.google.com/books/content?id=sa6pDAAAQBAJ&printsec=frontcover&img=1&zoom=5&edge=curl&source=gbs_api', 'thumbnail': 'http://books.google.com/books/content?id=sa6pDAAAQBAJ&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api'}, 'language': 'en', 'previewLink': 'http://books.google.fr/books?id=sa6pDAAAQBAJ&printsec=frontcover&dq=food&hl=&as_brr

Dans les données du premier livre, chercher où se trouve les informations suivantes (il peut s'agir de dictionnaires imbriqués) :
- le titre,
- le prix,
- la note (_averageRating_)

Afficher ces informations (cf la note ci-dessus pour accéder aux éléments d'un dictionnaire en python).

In [153]:
title_first_book = data_books[0].get("volumeInfo", "clé inexistante").get("title", "clé inexistante")
price_first_book = data_books[0].get("saleInfo", "clé inexistante").get("listPrice", "clé inexistante").get('amount', "clé inexistante")
average_rating_first_book = data_books[0].get("volumeInfo", "clé inexistante").get("averageRating", "clé inexistante")
print(f"Titre : {title_first_book}")
print(f"Prix : {price_first_book}")
print(f"Note : {average_rating_first_book}")

Titre : Food
Prix : 24.99
Note : clé inexistante


In [162]:
# Afficher les infos clés (price, title, rating) pour chaque livres de la liste data_books
list_data_books = [{"title":i.get("volumeInfo", "clé inexistante").get("title", "clé inexistante"),
                    "price":i.get("saleInfo", "clé inexistante").get("listPrice", "clé inexistante").get('amount', "clé inexistante"),
                    "rating":i.get("volumeInfo", "clé inexistante").get("averageRating", "clé inexistante")
                    } for i in data_books]
print(list_data_books)

[{'title': 'Food', 'price': 24.99, 'rating': 'clé inexistante'}, {'title': 'French Food France Food', 'price': 8.91, 'rating': 'clé inexistante'}, {'title': 'A History of Food', 'price': 28.99, 'rating': 'clé inexistante'}, {'title': 'On Food and Cooking', 'price': 19.49, 'rating': 4}, {'title': 'Introduction to Food Engineering', 'price': 78.02, 'rating': 5}, {'title': 'The Philosophy of Food', 'price': 26.82, 'rating': 'clé inexistante'}, {'title': 'Food Freezing', 'price': 94.94, 'rating': 'clé inexistante'}, {'title': 'Food, Energy, and Society', 'price': 65.4, 'rating': 4.5}, {'title': 'Proteomics for Food Authentication', 'price': 232.1, 'rating': 'clé inexistante'}, {'title': 'Hunger and Obesity', 'price': 37.6, 'rating': 'clé inexistante'}, {'title': 'Dictionary of Japanese Food', 'price': 12.65, 'rating': 4}, {'title': 'Food Processing Technology', 'price': 75.91, 'rating': 'clé inexistante'}, {'title': 'Introduction to Food Engineering', 'price': 66.45, 'rating': 5}, {'title'

Créer une liste de dictionnaire contenant les informations de chaque livre sous le format : 
```
[{
    "title": "title0",
    "price": price_0,
    "rating": rating_0
},
{
    "title": "title1",
    "price": price_1,
    "rating": rating_1
},
...
]
```

In [163]:
# Création d'une liste de dictionnaires pour les livres
books_list = [{"title":i.get("volumeInfo", "clé inexistante").get("title", "clé inexistante"),
                    "price":i.get("saleInfo", "clé inexistante").get("listPrice", "clé inexistante").get('amount', "clé inexistante"),
                    "rating":i.get("volumeInfo", "clé inexistante").get("averageRating", "clé inexistante")
                    } for i in data_books]

In [166]:
# Créer un dataframe à partir de la liste de dictionnaires
df_books = pd.DataFrame(books_list)
df_books


,title,price,rating
0,Food,24.99,clé inexistante
1,French Food France Food,8.91,clé inexistante
2,A History of Food,28.99,clé inexistante
3,On Food and Cooking,19.49,4
4,Introduction to Food Engineering,78.02,5
5,The Philosophy of Food,26.82,clé inexistante
6,Food Freezing,94.94,clé inexistante
7,"Food, Energy, and Society",65.40,4.5
8,Proteomics for Food Authentication,232.10,clé inexistante
9,Hunger and Obesity,37.60,clé inexistante


Filtrer les livres pour ne conserver que ceux ayant des valeurs pour les colonnes _price_ et _rating_.

- Possibilité d'utiliser la méthode pandas [_dropna_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)
- Afficher le dataframe et regarder ce qu'il s'est passé au niveau des index.
- Utiliser la méthode [_reset_index_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html) pour réinitialiser les index.

In [ ]:
# Filter les livres pour ne conserver que ceux ayant des valeurs pour les colonnes price et rating
df_books = df_books.dropna()
df_books = df_books[df_books.rating != "clé inexistante"]
df_books
# Reinitialiser les index du DataFrame
df_books.reset_index()

# Ajouter une colonne availability = False
df_books.assign("availability")


,title,price,rating
3,On Food and Cooking,19.49,4
4,Introduction to Food Engineering,78.02,5
7,"Food, Energy, and Society",65.40,4.5
10,Dictionary of Japanese Food,12.65,4
12,Introduction to Food Engineering,66.45,5
16,Encyclopedia of Food Microbiology,1672.00,5
18,"Food, Nutrition and Hygiene - According to NEP...",2.18,4
21,Food Of The Gods,11.99,5
24,The Food Of Love,1.99,5
25,Essentials of Food Science,56.19,5


---
## Partie 3 : Insertion des données en base

In [ ]:
# Utiliser sqlite3 pour insérer les données dans la base de données book_store dans la table book


In [ ]:
# Vérifier que le nombre de livres dans la table correspond au nb de livres scrapés + livres de l'API
